In [1]:
import itertools
import random
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path 
import sys
import threading
import os
from IPython.display import clear_output

In [2]:
YEARS = [str(year) for year in range(1990,2019)]

## Computer science authors in France
collaboration_df = pd.read_csv('myDATA/00-collaboration_df.csv')
collaboration_df

,ID,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
12908,8958327900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6508297663,0,0,0,0,0,0,0,0,0,...,4,7,7,8,8,8,8,8,8,8
103257,34571759000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,5,5,5
25815,7004267341,0,0,0,0,0,0,0,0,0,...,10,10,10,16,16,16,16,16,16,16
64536,8642393600,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12222,6507630481,0,0,0,0,0,0,0,0,0,...,18,18,18,18,18,29,29,29,29,29
193310,24577815500,0,0,0,0,0,0,0,0,0,...,4,4,4,4,6,13,16,16,16,70
36974,57195243976,0,0,0,0,0,0,0,0,0,...,0,3,3,3,3,3,3,3,8,8
12243,35328962100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,3


In [3]:
## retrieve the pubblication data(not collaborations)
file = '/home/leonardo/Desktop/PFE/Data/requested_data_on_authors.json'
d = json.load(open(Path(file)))

In [4]:
# Build a pandas dataframe from it
publications_df = pd.DataFrame.from_dict(d, orient='index')

In [5]:
# format it as callaboration_df
publications_df = publications_df.reset_index()
publications_df = publications_df.rename(columns={"index": "ID"})
publications_df['ID'] = publications_df['ID'].astype(int)

# keep only authors present in the collaborations dataframe
collab_IDs = collaboration_df["ID"]
publications_df = publications_df[publications_df.ID.isin(collab_IDs)]
publications_df

,ID,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
5,7003355588,2,2,2,1,4,0,5,5,0,...,7,4,4,15,11,7,11,9,8,6
29,56522848500,3,0,1,0,2,0,6,1,3,...,3,5,6,1,0,0,1,1,1,4
49,7004165433,5,1,1,2,10,5,6,2,6,...,4,3,11,7,6,10,6,3,3,4
79,6603870889,1,0,2,0,1,2,6,4,2,...,8,10,7,20,16,12,9,10,15,16
96,7005944861,10,10,3,7,8,8,4,15,9,...,9,8,12,10,20,19,17,12,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207020,57200496797,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2207039,15137130100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2207045,57196721826,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2207048,57196401698,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
## Find the starting collaboration year for each author
def get_ending_Y(aut):
    YEARS = [str(year) for year in range(1990,2019)]
    last_val = aut["2018"].values[0]
    for y in YEARS[::-1]:
        val = aut[y].values[0]
        if(val != 0):
            return y
        
# add starting collaboration year to each row of the dataset
def add_ending_y(df):
    df["ending_year"] = [0 for i in range(len(df))]
    i = 0
    for id in df["ID"]:
        
        if(i%500 == 0):
            print("done", i, "on", len(df), end='\r')
        i+=1
        
        aut = df.loc[df["ID"] == id]
        df.loc[df.ID == id, "ending_year"] = get_ending_Y(aut)
    return df

In [7]:
# Print the distribution of ending publication years for each author that has at least one collaboration
df = add_ending_y(publications_df)
for y in YEARS:
    num_auths = len(df.loc[df['ending_year']==y])
    print("for year ", y, "there are ", num_auths, "authors")

for year  1990 there are  447 authors
for year  1991 there are  473 authors
for year  1992 there are  589 authors
for year  1993 there are  843 authors
for year  1994 there are  909 authors
for year  1995 there are  1178 authors
for year  1996 there are  1552 authors
for year  1997 there are  1751 authors
for year  1998 there are  2112 authors
for year  1999 there are  2011 authors
for year  2000 there are  2271 authors
for year  2001 there are  2227 authors
for year  2002 there are  2783 authors
for year  2003 there are  3145 authors
for year  2004 there are  3499 authors
for year  2005 there are  4084 authors
for year  2006 there are  5056 authors
for year  2007 there are  5557 authors
for year  2008 there are  6683 authors
for year  2009 there are  7032 authors
for year  2010 there are  7366 authors
for year  2011 there are  7985 authors
for year  2012 there are  9396 authors
for year  2013 there are  10692 authors
for year  2014 there are  12520 authors
for year  2015 there are  14